In [2]:
from keras.preprocessing.image import ImageDataGenerator
import keras
import tensorflow as tf
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.layers import Conv2D, Lambda, MaxPooling2D, BatchNormalization, concatenate, GlobalAveragePooling2D, Dense
from keras.models import Model
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt 
import time

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_dataset = ImageDataGenerator(rescale=1./255)
test_dataset = ImageDataGenerator(rescale=1./255)
validation_dataset = ImageDataGenerator(rescale=1./255)


#BATCH_SIZE = 32
#IMG_SIZE = (160, 160)
#train_dir='Database/Train/'
#validation_dir='Database/Valid/'
#train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            #shuffle=True,
                                                            #batch_size=BATCH_SIZE,
                                                            #image_size=IMG_SIZE)
#validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                # shuffle=True,
                                                                # batch_size=BATCH_SIZE,
                                                                 #image_size=IMG_SIZE)
#import tensorflow as tf
#AUTOTUNE = tf.data.AUTOTUNE

#train_dataset= train_dataset.prefetch(buffer_size=AUTOTUNE)
#validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
#test_datagen = test_datagen.prefetch(buffer_size=AUTOTUNE)
 
batchsize = 10
train_generator=train_dataset.flow_from_directory('Databases/Train/', 
                                                  target_size=(224, 224), 
                                                  batch_size=batchsize, 
                                                  class_mode='categorical')
 
validation_generator = validation_dataset.flow_from_directory('Databases/Valid',
        target_size=(224, 224),
        batch_size=batchsize,
        class_mode='categorical',
        shuffle=False)
test_generator = test_dataset.flow_from_directory('Databases/Test/',
       target_size=(224, 224),
       batch_size=batchsize,
       class_mode='categorical',
       shuffle=False)

Found 358 images belonging to 2 classes.
Found 91 images belonging to 2 classes.
Found 223 images belonging to 2 classes.


# Model Creation
VGG Net: VGG network has simplest structure with 3 x 3 filter in each convolutional layer. There are two variant available VGG-16 and VGG1-9. In the image above VGG network is presneted.
Transfer Learning: The VGGNet pre-trained on ImageNet and Inception module for transfer learning, and trained the newly formed neural networks using new datasets. The approach combines the advantages of the VGGNet and Inception module.

In [4]:
#VGG19
base_model1=keras.applications.vgg19.VGG19(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
    classifier_activation='softmax'
)

#MobileNet
base_model2=tf.keras.applications.MobileNetV2(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
    classifier_activation='softmax'
)

#ResNet50 
base_model3 = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
    classifier_activation='softmax')



#VGG16
base_model4=keras.applications.vgg16.VGG16(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
    classifier_activation='softmax')

In [5]:
base_model1.summary()
base_model2.summary()
base_model3.summary()
base_model4.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## setting all layers as non-trainable

In [6]:
for layer in base_model1.layers:
    layer.trainable = False
    
for layer in base_model2.layers:
    layer.trainable = False 
    
for layer in base_model3.layers:
    layer.trainable = False
    
for layer in base_model4.layers:
    layer.trainable = False

## Defining custom activation function
As per the paper, custom activation function Swish is used in place of Relu in the model.

In [7]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
def swish(x):
    return (K.sigmoid(x) * x)
get_custom_objects().update({'swish': Activation(swish)})

## Defining convolution batch normalization function with Swish activation

In [8]:
def Conv2d_BN(x, nb_filter,kernel_size, padding='same',strides=(1,1)):   
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides)(x)
    x = Lambda(swish)(x)
    x = BatchNormalization(axis=3)(x)  
    return x

## Defining Inception Block:
Inception Modules are used in Convolutional Neural Networks to allow for more efficient computation and deeper Networks through a dimensionality reduction with stacked 1×1 convolutions. The modules were designed to solve the problem of computational expense, as well as overfitting, among other issues.

In [9]:
def Inception(x,nb_filter):  
    branch1x1 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1))  
  
    branch3x3 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1))  
    branch3x3 = Conv2d_BN(branch3x3,nb_filter,(3,3), padding='same',strides=(1,1))  
  
    branch5x5 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1))  
    branch5x5 = Conv2d_BN(branch5x5,nb_filter,(3,3), padding='same',strides=(1,1))
    branch5x5 = Conv2d_BN(branch5x5,nb_filter,(3,3), padding='same',strides=(1,1))
  
    branchpool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)  
    branchpool = Conv2d_BN(branchpool,nb_filter,(1,1),padding='same',strides=(1,1))  
  
    x = concatenate([branch1x1,branch3x3,branch5x5,branchpool],axis=3)    
    return x

## Adding new layers in model architechture
Global Average Pooling
Fully connected layers are replaced by a global pooling layer to conduct the dimension reduction of feature maps.

In [10]:
# MODEL 1- VGG19

x = base_model1.layers[-6].output


x = Conv2d_BN(x,512,(3,3),strides=(1,1),padding='same')  
x = MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')(x) 

x = Inception(x,512)  
x = Inception(x,512)  

x = GlobalAveragePooling2D()(x)
predictions_1 = Dense(2, activation='softmax')(x)
model1 = Model(inputs=base_model1.input, outputs=predictions_1)

In [11]:
# MODEL 2 MobileNet

x = base_model2.layers[-6].output


x = Conv2d_BN(x,512,(3,3),strides=(1,1),padding='same')  
x = MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')(x) 

x = Inception(x,512)  
x = Inception(x,512)  

x = GlobalAveragePooling2D()(x)
predictions_2 = Dense(2, activation='softmax')(x)
model2 = Model(inputs=base_model2.input, outputs=predictions_2)

In [12]:
 #ResNet50
x = base_model3.layers[-6].output


x = Conv2d_BN(x,512,(3,3),strides=(1,1),padding='same')  
x = MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')(x) 

x = Inception(x,512)  
x = Inception(x,512)  

x = GlobalAveragePooling2D()(x)
predictions_3 = Dense(2, activation='softmax')(x)
model3 = Model(inputs=base_model3.input, outputs=predictions_3)

In [13]:
#VGG-16
x = base_model4.layers[-6].output


x = Conv2d_BN(x,512,(3,3),strides=(1,1),padding='same')  
x = MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')(x) 

x = Inception(x,512)  
x = Inception(x,512)  

x = GlobalAveragePooling2D()(x)
predictions_4 = Dense(2, activation='softmax')(x)
model4 = Model(inputs=base_model4.input, outputs=predictions_4)

In [14]:
model1.summary()
model2.summary()
model3.summary()
model4.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

## Compiling the Model 

In [15]:
def comp():
    learning_rate=0.0001
    decay=1e-6
    momentum=0.8
    nesterov=True
    sgd_optimizer= SGD(lr = learning_rate, decay = decay, momentum = momentum, nesterov = nesterov)
    models=['VGG-19','MobileNet','ResNet','VGG-16']
    
    for i in models:
        if i=='VGG-19':
            model1.compile(loss='categorical_crossentropy',optimizer=sgd_optimizer,metrics=['accuracy','mse'])
        elif i=='MobileNet':
            model2.compile(loss='categorical_crossentropy',optimizer=sgd_optimizer,metrics=['accuracy','mse'])
        elif i=='ResNet':
            model3.compile(loss='categorical_crossentropy',optimizer=sgd_optimizer,metrics=['accuracy','mse'])
        else:
            model4.compile(loss='categorical_crossentropy',optimizer=sgd_optimizer,metrics=['accuracy','mse'])
        
if __name__ == "__main__":
    comp()

## Training the Models

In [19]:
#Model1 VGG-19
import time
epochs =40
batch_size = 10

start = time.time()
history_1 = model1.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=epochs, batch_size =batch_size,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
end = time.time()
print("Execution Time : ", (end - start))

Epoch 1/40
36/35 [==============================] - 119s 3s/step - loss: 0.6049 - accuracy: 0.6732 - mse: 0.2090 - val_loss: 0.6806 - val_accuracy: 0.7802 - val_mse: 0.2437
Epoch 2/40
36/35 [==============================] - 116s 3s/step - loss: 0.4744 - accuracy: 0.7905 - mse: 0.1524 - val_loss: 0.7012 - val_accuracy: 0.2747 - val_mse: 0.2540
Epoch 3/40
36/35 [==============================] - 112s 3s/step - loss: 0.4112 - accuracy: 0.8212 - mse: 0.1279 - val_loss: 0.7260 - val_accuracy: 0.2198 - val_mse: 0.2664
Epoch 4/40
36/35 [==============================] - 112s 3s/step - loss: 0.3762 - accuracy: 0.8492 - mse: 0.1155 - val_loss: 0.7603 - val_accuracy: 0.2198 - val_mse: 0.2835
Epoch 5/40
36/35 [==============================] - 112s 3s/step - loss: 0.3469 - accuracy: 0.8603 - mse: 0.1055 - val_loss: 0.7849 - val_accuracy: 0.2198 - val_mse: 0.2957
Epoch 6/40
36/35 [==============================] - 112s 3s/step - loss: 0.3276 - accuracy: 0.8743 - mse: 0.0987 - val_loss: 0.7766 - v

In [16]:
#MobileNet
epochs =40
batch_size = 10
import time
start = time.time()
history_2 = model2.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=epochs, batch_size =batch_size,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
end = time.time()
print("Execution Time : ", (end - start))

Epoch 1/40
36/35 [==============================] - 20s 562ms/step - loss: 0.7007 - accuracy: 0.5726 - mse: 0.2512 - val_loss: 0.5744 - val_accuracy: 0.7363 - val_mse: 0.1926
Epoch 2/40
36/35 [==============================] - 22s 609ms/step - loss: 0.4822 - accuracy: 0.7933 - mse: 0.1551 - val_loss: 0.6214 - val_accuracy: 0.7473 - val_mse: 0.2151
Epoch 3/40
36/35 [==============================] - 25s 690ms/step - loss: 0.3888 - accuracy: 0.8631 - mse: 0.1167 - val_loss: 0.6555 - val_accuracy: 0.6264 - val_mse: 0.2324
Epoch 4/40
36/35 [==============================] - 24s 673ms/step - loss: 0.3389 - accuracy: 0.8966 - mse: 0.0986 - val_loss: 0.6511 - val_accuracy: 0.6154 - val_mse: 0.2316
Epoch 5/40
36/35 [==============================] - 22s 620ms/step - loss: 0.3308 - accuracy: 0.8771 - mse: 0.0978 - val_loss: 0.5853 - val_accuracy: 0.6703 - val_mse: 0.2025
Epoch 6/40
36/35 [==============================] - 22s 619ms/step - loss: 0.2838 - accuracy: 0.9106 - mse: 0.0789 - val_loss

In [17]:
#ResNet50
epochs =40
batch_size = 10
import time
start = time.time()
history_3 = model3.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=epochs, batch_size =batch_size,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
end = time.time()
print("Execution Time : ", (end - start))

Epoch 1/40
36/35 [==============================] - 38s 1s/step - loss: 0.6820 - accuracy: 0.6341 - mse: 0.2445 - val_loss: 0.7199 - val_accuracy: 0.2198 - val_mse: 0.2634
Epoch 2/40
36/35 [==============================] - 40s 1s/step - loss: 0.6514 - accuracy: 0.7514 - mse: 0.2292 - val_loss: 0.7627 - val_accuracy: 0.2198 - val_mse: 0.2846
Epoch 3/40
36/35 [==============================] - 39s 1s/step - loss: 0.6277 - accuracy: 0.7793 - mse: 0.2176 - val_loss: 0.8217 - val_accuracy: 0.2198 - val_mse: 0.3134
Epoch 4/40
36/35 [==============================] - 40s 1s/step - loss: 0.6089 - accuracy: 0.8045 - mse: 0.2085 - val_loss: 0.8995 - val_accuracy: 0.2198 - val_mse: 0.3500
Epoch 5/40
36/35 [==============================] - 41s 1s/step - loss: 0.5891 - accuracy: 0.8101 - mse: 0.1990 - val_loss: 0.9953 - val_accuracy: 0.2198 - val_mse: 0.3926
Epoch 6/40
36/35 [==============================] - 40s 1s/step - loss: 0.5678 - accuracy: 0.8101 - mse: 0.1891 - val_loss: 1.1028 - val_acc

In [18]:
#VGG-16
epochs =40
batch_size = 10
import time
start = time.time()
history_4 = model4.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=epochs, batch_size =batch_size,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
end = time.time()
print("Execution Time : ", (end - start))

Epoch 1/40
36/35 [==============================] - 101s 3s/step - loss: 0.6486 - accuracy: 0.6313 - mse: 0.2286 - val_loss: 0.6255 - val_accuracy: 0.7802 - val_mse: 0.2164
Epoch 2/40
36/35 [==============================] - 100s 3s/step - loss: 0.5048 - accuracy: 0.7793 - mse: 0.1633 - val_loss: 0.6174 - val_accuracy: 0.7802 - val_mse: 0.2124
Epoch 3/40
36/35 [==============================] - 101s 3s/step - loss: 0.4557 - accuracy: 0.7961 - mse: 0.1447 - val_loss: 0.6321 - val_accuracy: 0.7802 - val_mse: 0.2196
Epoch 4/40
36/35 [==============================] - 101s 3s/step - loss: 0.4043 - accuracy: 0.8492 - mse: 0.1241 - val_loss: 0.6785 - val_accuracy: 0.6264 - val_mse: 0.2427
Epoch 5/40
36/35 [==============================] - 101s 3s/step - loss: 0.3778 - accuracy: 0.8464 - mse: 0.1142 - val_loss: 0.7062 - val_accuracy: 0.2857 - val_mse: 0.2566
Epoch 6/40
36/35 [==============================] - 102s 3s/step - loss: 0.3652 - accuracy: 0.8408 - mse: 0.1112 - val_loss: 0.7486 - v

In [20]:
model1.save_weights('mix1_1epochs.h5')
model2.save_weights('mix2_1epochs.h5')
model3.save_weights('mix3_1epochs.h5')
model4.save_weights('mix4_1epochs.h5')

In [21]:
import time
start=time.time()
test_output_1= model1.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
end=time.time()
print(model2.metrics_names)
print(test_output_1)
print("testing time : "+ str(end-start))

23/22 [==============================] - 30s 1s/step - loss: 0.3092 - accuracy: 0.8744 - mse: 0.0937
['loss', 'accuracy', 'mse']
[0.3092218339443207, 0.8744394779205322, 0.09369439631700516]
testing time : 31.295392274856567


In [22]:
start=time.time()
test_output_2= model2.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
end=time.time()
print(model1.metrics_names)
print(test_output_2)
print("testing time : "+ str(end-start))

23/22 [==============================] - 4s 179ms/step - loss: 0.1958 - accuracy: 0.9283 - mse: 0.0592
['loss', 'accuracy', 'mse']
[0.1957830935716629, 0.9282511472702026, 0.05922241508960724]
testing time : 4.413250207901001


In [23]:
start=time.time()
test_output_3= model3.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
end=time.time()
print(model1.metrics_names)
print(test_output_3)
print("testing time : "+ str(end-start))

23/22 [==============================] - 9s 408ms/step - loss: 3.8348 - accuracy: 0.1031 - mse: 0.8608
['loss', 'accuracy', 'mse']
[3.8348042964935303, 0.1031390130519867, 0.8607578277587891]
testing time : 9.922276973724365


In [24]:
start=time.time()
test_output_4= model4.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
end=time.time()
print(model1.metrics_names)
print(test_output_4)
print("testing time : "+ str(end-start))

23/22 [==============================] - 26s 1s/step - loss: 0.3130 - accuracy: 0.8700 - mse: 0.0925
['loss', 'accuracy', 'mse']
[0.31299126148223877, 0.8699551820755005, 0.09249529987573624]
testing time : 26.95220446586609


In [25]:
import pandas as pd
data={'Accuracy':[test_output_1[1],test_output_2[1],test_output_3[1],test_output_4[1]],
     'Loss':[test_output_1[0],test_output_2[0],test_output_3[0],test_output_4[0]],
     'MSE':[test_output_1[2],test_output_2[2],test_output_3[2],test_output_4[2]]}
models=['VGG-19','MobileNet v2','ResNet50','VGG-16']
df=pd.DataFrame(data,index=models)
df

,Accuracy,Loss,MSE
VGG-19,0.874439,0.309222,0.093694
MobileNet v2,0.928251,0.195783,0.059222
ResNet50,0.103139,3.834804,0.860758
VGG-16,0.869955,0.312991,0.092495
